In [ ]:
import os
import csv
from countries_languages import country_to_language
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime
from functools import reduce
from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')

#leagues value
leagues_value = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')
#leagues value large 
leagues_value_large = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')


In [ ]:
#BIG DATASET

large_df_combined = load_csv_dataset('updated COMBINED DF_WORKING.csv')


terrell_new_df = load_csv_dataset('CSVs of edge cases - for T/updated Terrell new DF.csv')
len(terrell_new_df), len(large_df_combined)


In [ ]:
#Group 1 

group_1_updated = load_csv_dataset('updated_Group1_fixes_complete_ii.csv')
len(group_1_updated)

In [ ]:
#Group 2 

online_lookup_group_2 = load_csv_dataset('Group 2 (for T) - Find their $ Info/Online lookup required - 1 name found.csv')

lookup_group_2 = load_csv_dataset('Group 2 (for T) - Find their $ Info/lookup_required_matches.csv')

cyrillic_group_2 = load_csv_dataset('Group 2 (for T) - Find their $ Info/Cyrillic_Found_Guys.csv')
len(online_lookup_group_2), len(lookup_group_2), len(cyrillic_group_2)

In [ ]:
#Group 3 

cyrillic_group_3 = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/Cyrillic_Guys_Nobody_Found.csv')

zeros_group_3 = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/odd_name_found_0_cases.csv')

online_lookup_group_3 = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/Online lookup required - 0 names found.csv')

len(cyrillic_group_3), len(zeros_group_3), len(online_lookup_group_3)

In [ ]:
leagues_value

In [ ]:
# Other group

wrong_df = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated Terrell lookup cases - wrong DF.csv')
wrong_df = wrong_df.drop(columns={'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'})
lookup_required_dudes = load_csv_dataset('CSVs of edge cases - for F/"other" group/lookup_required_dudes.csv')

large_dataset_said_0 = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated_large_dataset_said_0.csv')

match_date_filter_said_0_guys = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated match date said zero guys.csv')

len(wrong_df), len(lookup_required_dudes), len(large_dataset_said_0), len(match_date_filter_said_0_guys)

## WORKING WITH WRONG DF

In [ ]:
from fuzzywuzzy import process

def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

#threshold match one 
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}


def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''
    

def check_tokens(this_jersey, match):
    jersey_tokens = this_jersey.lower().split()
    match_tokens = match.lower().split()
    
    if all(token in match_tokens for token in jersey_tokens):
        return True
    elif all(token in jersey_tokens for token in match_tokens) and len(jersey_tokens) == len(match_tokens) + 1:
        return True
    elif all(token in match_tokens for token in jersey_tokens) and len(jersey_tokens) > len(match_tokens):
        return True
    else:
        return False

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}
        #print(match_token_start_chars, f"*{jersey_tokens}")


        #for token in jersey_tokens:
         #   print(token, token[0] in match_token_start_chars)

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches

In [ ]:
wrong_df['New Found Name'] = ''
wrong_df['Status II'] = ''

In [ ]:
wrong_df[wrong_df['Status'] == 'SUCCESS']['Lookup Return Case'].value_counts()

In [ ]:
success_status_counter = 0
for i in range(0, 1063):
    names_found = []
    num_identical_rows = 0
    num_names_this = 0
    index = i
    row = wrong_df.loc[index]
    # Extract relevant columns
    match = row['Match']
    date = row['Date']
    if type(row['Found Name']) == str:
        if row['Found Name'].startswith('['):
            FOUND_NAME = eval(row['Found Name']) 
        else:
            FOUND_NAME = row['Found Name']
    
    if row['Status II'] != 'Done':
    
    if row['Status'] == 'SUCCESS':
        #for all the success guys. 
        #these are all people that eventually T needs to use imputing methods.             
        NAMES_FOUND = row['Name(s) Found']

        ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
        this_country_code = row['Team Country Code']
        if pd.isna(this_country_code):
            country_name = 'Namibia'
        else:
            country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
        dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()
        if 'Mohd' in ORIGINAL_JERSEY:
            ORIGINAL_JERSEY = ORIGINAL_JERSEY.replace('Mohd', 'Mohammed')
        if type(FOUND_NAME) == list:
            FOUND_NAME.append(NAMES_FOUND)
            found_names = FOUND_NAME
        else:
            found_names = [FOUND_NAME, NAMES_FOUND]
        closest_name = filter_multi_word_matches_by_jersey_tokens([token[0] for token in ORIGINAL_JERSEY.split(' ')], found_names)
        if len(closest_name) == 1:
            0==0
            print(i, ORIGINAL_JERSEY, [name for name in closest_name if name in found_names]) #closest_name, found_names
            success_status_counter += 1
            wrong_df.at[i, 'New Found Name'] = [name for name in closest_name if name in found_names][0]
            wrong_df.at[i, 'Status II'] = 'Done'

            #wrong_df['Status 2'] MAKE A NEW STATUS TO MARK THEM OFF ONCE DONE  
        else:
            0==0
            #print(i, ORIGINAL_JERSEY, closest_name, found_names)
            #print(i, f"threshold match says {threshold_player_match(ORIGINAL_JERSEY, found_names)}")

In [ ]:
wrong_df[wrong_df['Status II'] == 'Done']

In [ ]:
wrong_df

In [ ]:
found_names



In [ ]:
wrong_df

#do the threshold match. if its either Name(s) found column or Found Name column use that name 